In [164]:
## Importing Libraries ##
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import openpyxl
from datetime import datetime
import copy
from scipy import stats

In [165]:
## Importing dataset ##
df = pd.read_csv('event_dataset.csv')
df

,Unnamed: 0,OBS_TIME_LOC,PRCP,VOID_FLAG,LOC,LOC_NAME,LAT,LONG
0,0,2007-11-07 10:00:00,0.2,N,58216,Cape Byron Aws,-28.6399,153.6358
1,1,2007-11-07 11:00:00,2.8,N,58216,Cape Byron Aws,-28.6399,153.6358
2,2,2007-11-07 12:00:00,3.8,N,58216,Cape Byron Aws,-28.6399,153.6358
3,3,2007-11-07 13:00:00,7.2,N,58216,Cape Byron Aws,-28.6399,153.6358
4,4,2007-11-07 14:00:00,9.6,N,58216,Cape Byron Aws,-28.6399,153.6358
...,...,...,...,...,...,...,...,...
689957,88763,2022-07-31 12:00:00,0.0,N,558053,Main Arm,-28.5028,153.3833
689958,88764,2022-07-31 15:00:00,0.0,N,558053,Main Arm,-28.5028,153.3833
689959,88765,2022-07-31 18:00:00,0.0,N,558053,Main Arm,-28.5028,153.3833
689960,88766,2022-07-31 21:00:00,0.0,N,558053,Main Arm,-28.5028,153.3833


In [166]:
## Setting time values to datetime ##
df.OBS_TIME_LOC = pd.to_datetime(df['OBS_TIME_LOC'])

In [167]:
## Putting values in pivot table and removng outliers ##
result = pd.DataFrame()

## Looping over each location, and manipulating values ##
for i in df.LOC_NAME.unique():
    df_loc = df[df.LOC_NAME == str(i)]
    df_loc = df_loc.set_index('OBS_TIME_LOC')
    df_loc['diff'] = df_loc['PRCP'].diff(periods=1)
    df_loc['diff'] = df_loc['diff'].bfill().ffill()
    df_loc['z'] = np.abs(stats.zscore(df_loc['diff']))
    df_loc = df_loc.mask(df_loc['z'] > 3, np.nan)
    resampled = df_loc.resample('D').median()
    
    ## Merging all dataframes ##
    result = pd.concat([result,resampled.PRCP.rename(str(i))],axis=1)
    
## Removing all outliers ##    
result = result[result.columns[result.isna().sum() < 2000]]

## Droping na values ##
result = result.dropna()

In [168]:
## Creating a CSV file ## 
result.to_csv('cleaned_event_dataset.csv')